In [ ]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collection as c #Rename collection package
from datetime import datetime, timedelta


In [ ]:
#Comvert csv file to DataFrame object
df = pd.read_csv('denue_inegi_02_.csv', encoding='latin-1')

#Print df info
df.info()

#Print df head
df.head(3)

### Let's talk about Tijuana...

Once we have seen a brief overview of the data provided, lets see what more we can extract from it, our main focus will be Tijuana.

Here, we would try to ask the following questions:

1. **What is the zone that has more employability in TJ?**
2. **What is the most representative industry in TJ?**
3. **What kind of businesses have survived more than 10 years?**
4. **What are the top 3 businees by sector, in terms of seniority and size (How many people is working in the business)**



**1. What is the zone that has more employability in TJ? (Done)**

First, we will start by defining our subset TJ.

In [ ]:
#Filter dataset
tj = df[df['municipio'] == 'Tijuana'].copy()

#Print tj info
tj.info()

#Print first observation of tj Dataframe
tj.head(1)

We can see that we have various types of variables, from which many are unnecesary in our analysis, lets start by dropping them!

In [ ]:
#Drop unnecesary columns (latitude, coordinates, etc...)
tj.drop(columns=c.columns_to_drop, inplace=True)

#Print tj DataFrame
tj.info()


As not exist an explicit column for assess the employability in TJ, we will use what we have at hand.

First, we can see that we have some interesting columns:
1. per_ocu (giving mention to 'personal ocupado', which tells us how many people are working in the company)
2. localidad (zones where the business are located in)

given that we cannot perform a direct relation operation because neither 'per_ocu' nor 'localidad' are numeric (both are categorical), we must have to transform at least one variable to make it a numerical type

so for this, we have to re-scale the 'per_ocu' variable, starting with the companies with '0 to 5' employees with value 1 (minimum possible) to companies with '251+' employees with value 251 (minimum possible). Fortunately, we've already created a dictionary that contains the key-values for this variable, this dictionary is available in collection.py. Let's work!

In [ ]:
#Create a column called 'minimum_employees' with lambda filtration
tj['minimum_employees'] = tj['per_ocu'].apply(lambda x: c.cat_to_num_perocu.get(x))

#Print all unique values in 'minimum_employees'
tj['minimum_employees'].unique()

Once we have done that, we can group by 'localidad' and filter only the top 5 zones with minimum employees.

In [ ]:
#Grouping by manzana
zone_minemployee = tj.groupby('localidad')['minimum_employees'].sum().reset_index()

#Sort minimum employees values
zone_minemployee = zone_minemployee.sort_values(by='minimum_employees', ascending=False)

#Select top5zones_1
top5zones_1 = zone_minemployee.head(5)

#Print top5zones_1
top5zones_1

It seems like Tijuana is the zone added by default, given that we want to create a barplot, it would be better to drop that column, let's do it

In [ ]:
#Create top10zones_1 DataFrame subseting all zones != 'Tijuana' in zone_minemployee
top5zones_1 = zone_minemployee.loc[zone_minemployee['localidad'] != 'Tijuana'].head(5)

#Print top10zones_1 DataFrame
top5zones_1

Now, lets plot the top 10 zones with minimum employees, or, in other words, the 5 zones with more chances to find more employees

In [ ]:
#Definine barplot
ax = sns.barplot(data = top5zones_1, x = 'localidad', y = 'minimum_employees')

# Define custom labels for the x-axis
custom_x_labels = ['Villa del Campo', 'Thomas Alva Edison [PI]', 'El Niño', 'Cuesta Blanca', 'Toyota']

# Set custom labels for the x-axis
ax.set_xticklabels(custom_x_labels)

# Custom labels
ax.set_ylabel('Num of Employees')
ax.set_xlabel(' ')

#rotate labels
plt.xticks(rotation=90)


The previous visualization reveals that the **top 5 zones** with the **highest number of minimum employees** in the locality of Tijuana are:

1. **Villa Del Campo**
2. **Thomas Alva Edison** (Industrial Zone)
3. **El Niño**
4. **Casa Blanca**
5. **Toyota** (Manufacture Company - Industrial Zone)

**We must be cautious about the information provided earlier, as this information was collected in the 2022. Also, recall that lot of observations were filtered, leaving out those observations with 'Tijuana' as locality.**

2. **What is the most representative industry in TJ?**

To adress this question, we must perform an analysis in those columns that gives us information about the industry that each business belongs to, if we take a brief look at the 'tj' dataframe we can see that we have some interesting columns that mey serves us to get good conclusions about the raised question. This columns are:

1. nombre_act
2. nom_estab
3. minimum_employees



First, lets take a look at the 'nombre_act' column, in this way we can see the state of the data

In [ ]:
#Print the 'nombre_act' dataframe
tj['nombre_act'].head(12)

As we can see, the 'nombre_act' column has a description of the business, more specifically the sector to which the business fits in.

this could be useful if we wanted to perform a deep analysis on each seactor, but, as we dont want to, we will only select the most representative word, that is, the first word of each observation and then save that element in a variable called 'sector' (tj['sector]). 
Lets do it!.

In [84]:
#Filter 'nombre_act' column by spliting each string and selecting the first element (word)
tj['sector'] = tj['nombre_act'].apply(lambda x: x.split()[0])

#Print the first 21 rows
tj['sector'].head(10)

52        Servicios
60        Beneficio
61        Beneficio
77        Servicios
79            Pesca
83        Beneficio
84            Otros
92     Piscicultura
228           Pesca
291       Beneficio
Name: sector, dtype: object

Great, it seems that we've already filtered the first element and saved it in 'sector' column. Now, lets select the 5 most representative businesses sectors in Tijuana.

In [85]:
#select the sum of the minimum employees in each business and group by sector
industry_employees = tj.groupby('sector')['minimum_employees'].sum().reset_index()

#Sort values in ascending, save the first 5 observations
top5_2 = industry_employees.sort_values(by='minimum_employees', ascending=False).head(5)

#Print the top5_2 df
top5_2

,sector,minimum_employees
60,Fabricación,77717
38,Comercio,72156
57,Escuelas,24245
123,Servicios,21525
119,Restaurantes,18255


This looks fantastic, but, we will improve it performing a visualization of the insight!

In [ ]:
#Definine barplot
ax = sns.barplot(data = top5_2, x = 'sector', y = 'minimum_employees')

#Custom set title
ax.set_title('Min. Num of Employees by Sector')

# Custom labels
ax.set_ylabel('Employees')
ax.set_xlabel(' ')

#rotate labels
plt.xticks(rotation=90)

3. **What type of businesses that have survived more than 10 years?**

To solve this question we will have to make use of two vaiables:

1. fecha_alta
2. tj_sector (variable assigned previously by us)

First, we have to create a subset of tj by filtering those business with 10 years older or more, in order to do that, we must first have to check if the fecha_alta variable is a date time column.

In [ ]:
#Display fecha_alta type
print(tj['fecha_alta'].info())

We can see that this column is not a datetime data type. Let's convert it to datetime data type!.

In [ ]:
#Convert 'fecha_alta' data type to date_time object
tj['fecha_alta'] = pd.to_datetime(tj['fecha_alta'])

#Check if the operation has been done correctly
tj['fecha_alta']

Perfect, it seems that now is possible to perform a subset with all business with 10 years or more older. 

In [ ]:
#Create a subset 'older business' dataframe by filtering all the business that are 10 years old or older
older_business = tj.loc[tj['fecha_alta'] <= '2012-01-01']

#Preview subset
older_business

Now, once we have performed the filtering, we can group by sector and count how many business are living since 10 years ago, then, assgin the top 5 more reliables sectors in a variable called top5_3


In [ ]:
#Group by sector and count all the business for each one
top5_3 = older_business['sector'].value_counts().head(5)

top5_3 = pd.DataFrame(top5_3).reset_index()

top5_3.columns = ['sector', 'count']

#Display top5_3
display(top5_3)

Great, we found what are the top 5 sectors that have survived along 10 years since 2012, lets visualize our findings and give a resume about our insights.

In [ ]:
#Definine barplot
ax = sns.barplot(data = top5_3, x='sector', y='count')

#Custom title
ax.set_title('Business that have Survived +10 years')

# Custom labels
ax.set_ylabel('Count')
ax.set_xlabel(' ')

#rotate labels
plt.xticks(rotation=90)

4. **What are the top 3 businees by sector, in terms of seniority and size (How many people is working in the business) ?**

Finally, we reach the final question to solve, to address this question we can use the variable 'sector' and the recently created 'older_business' dataframe. In order to do that, we have to create a dicitonary where each key represent a sector and each value a list with the 3 business with more employees from that sector. 
This businesses will have to be all old business (+10 years). Lets do it!

In [89]:
#Initialize dictionary where elements will be stored
top_business_by_sector = {}


#Create a list for each unique element in sector column
sectors = older_business['sector'].unique()

#Create the dictionary
for sector in sectors:
    #Create local variables 
    list_tuples = []
    list_business = []

    #Adding top 3 business in tuples list
    list_tuples = older_business.loc[older_business['sector'] == sector, ['raz_social']]\
                            .value_counts(ascending=False)\
                            .head(3)\
                            .index.tolist()
    
    #Converting tuples to list format
    for element in list_tuples:
        list_business.append(list(element)[0])

    #Addinf list of top business by sector
    top_business_by_sector[sector] = list_business

    #Filtering those list with len=0
    if len(top_business_by_sector[sector]) == 0:
        del top_business_by_sector[sector]

#Review the dictionary recently created
top_business_by_sector

{'Otros': ['HIPODROMO DE AGUA CALIENTE SA DE CV',
  'GOBIERNO DEL ESTADO ',
  'TOTAL LOGISTICS S DE RL DE CV'],
 'Minería': ['PETREMEX SA DE CV'],
 'Captación,': ['COMISION ESTATAL DE SERVICIOS PÚBLICOS DE TIJUANA '],
 'Generación': ['COMISION FEDERAL DE ELECTRICIDAD '],
 'Edificación': ['A CONSTRUCTORES S DE RL DE CV',
  'INGENIERIA CONSTRUCCION Y AIRE ACONDICIONADO SA DE CV',
  'INMOBILIARIA Y DESARROLLADORA ENCASA SA DE CV'],
 'Instalaciones': ['AIRE ACONDICIONADO DEL NOROESTE ARP SA DE CV',
  'AIRE TEC DE MEXICO S DE RL DE CV',
  'AJ CONSTRUCCIONES INDUSTRIALES S DE RL DE CV'],
 'Construcción': ['ALBERCAS VALLARTA SA DE CV',
  'JAPAL INGENIERÍA SA DE CV',
  'URBANIZADORA ROMA '],
 'Otras': ['AUTOSERVICIO TAURO ',
  'IMPULSO COMERCIAL NACIONAL SA DE CV',
  'TJ FRAMES & MOLDINGS SA DE CV'],
 'Montaje': ['COMERCIAL DE ESTRUCTURAS METALICAS SA DE CV',
  'CONCRETOS PRESFORZADOS DE BAJA CALIFORNIA SA DE CV',
  'MARCAL CREACION ESTRUCTURAL SA DE CV'],
 'Trabajos': ['GEOCIMENTACIONES DE TI

We can realize that for each key we have a list of 